# Import Library 

In [1]:
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_hub as hub
import tensorflow_model_analysis as tfma 
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner, Evaluator, Pusher
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing
import IPython
import os


c:\Users\ASUS\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


# Set Variable

In [2]:
PIPELINE_NAME = "resto-pipeline"
SCHEMA_NAME = "resto-tfdv-schema"
PIPELINE_SOURCE = os.path.join('faisal_ag_037-pipeline', PIPELINE_NAME)
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)


In [3]:
DATA_PATH = "dataset"

In [4]:
Interact = InteractiveContext(pipeline_root=PIPELINE_SOURCE)

# Data Ingestion

Data ingestion adalah proses pengumpulan dan memasukkan data ke dalam sistem pemrosesan atau penyimpanan untuk digunakan dalam model Machine Learning.

## ExampleGen

ExampleGen adalah komponen yang digunakan untuk membaca data mentah dari sumber data (seperti file CSV, database, dll.) dan mengubahnya menjadi format yang dapat digunakan oleh pipeline Machine Learning. Ini adalah langkah awal untuk memastikan bahwa data yang akan diproses selanjutnya tersedia dalam format yang konsisten.

In [5]:
eg_output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)

In [6]:
exampleGen = CsvExampleGen(input_base=DATA_PATH, output_config=eg_output)

In [7]:
Interact.run(exampleGen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 24
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

# Data Validation

Data validation bertujuan untuk memastikan bahwa data yang digunakan dalam model Machine Learning memenuhi kualitas dan format yang diperlukan, serta untuk mengidentifikasi anomali atau inkonsistensi dalam data.

## StatisticGen

StatisticGen adalah komponen yang digunakan untuk menghitung statistik deskriptif dari data, seperti rata-rata, median, dan distribusi frekuensi. Statistik ini berguna untuk memahami karakteristik data yang digunakan secara statistik.

In [8]:
statisticsGen = StatisticsGen(
    examples=exampleGen.outputs["examples"]
)

In [9]:
Interact.run(statisticsGen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 25
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [10]:
Interact.show(statisticsGen.outputs["statistics"])

## SchemaGen

SchemaGen secara otomatis menghasilkan skema data berdasarkan statistik yang dihitung oleh StatisticGen. Skema ini menggambarkan tipe data yang diharapkan dan rentang nilai yang valid untuk setiap fitur, yang membantu dalam mendeteksi perubahan atau anomali dalam data di masa mendatang.

In [11]:
schemaGen = SchemaGen(statistics=statisticsGen.outputs["statistics"])


In [12]:
Interact.run(schemaGen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 26
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [13]:
Interact.show(schemaGen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'label',INT,required,,-
'sentence',BYTES,required,,-


## ExampleValidator

ExampleValidator memvalidasi data contoh yang ada terhadap skema yang dihasilkan oleh SchemaGen. Tujuannya adalah untuk mengidentifikasi dan menandai contoh data yang tidak sesuai dengan skema yang telah ditetapkan, seperti data yang hilang atau tipe data yang salah.

In [14]:
exampleValidator = ExampleValidator(
    statistics=statisticsGen.outputs['statistics'],
    schema=schemaGen.outputs['schema']
)

In [15]:
Interact.run(exampleValidator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 27
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [16]:
Interact.show(exampleValidator.outputs['anomalies'])

# Data Preprocessing

Data preprocessing adalah proses transformasi dan pembersihan data mentah untuk meningkatkan kualitas data sebelum digunakan dalam model Machine Learning.

## Transform

Transform adalah komponen yang digunakan untuk mengubah atau memanipulasi data mentah menjadi format yang lebih sesuai untuk pelatihan model. Ini bisa termasuk normalisasi, penskalaan, encoding kategori, dan operasi transformasi lainnya. Proses ini penting untuk memastikan bahwa data siap dan optimal untuk digunakan dalam model.

In [17]:
TRANSFORM_FILE = "resto-transform.py"

In [18]:
%%writefile {TRANSFORM_FILE}

import tensorflow as tf

LABEL = "label"
SENTENCE = "sentence"

def transformed_name(key):
    return key + "_xf"

def preprocessing_fn(inputs):
    outputs = {}
    outputs[transformed_name(SENTENCE)] = tf.strings.lower(inputs[SENTENCE])
    outputs[transformed_name(LABEL)] = tf.cast(inputs[LABEL], tf.int64)
    return outputs

Overwriting resto-transform.py


In [19]:
transform  = Transform(
    examples= exampleGen.outputs['examples'],
    schema= schemaGen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_FILE)
)

In [20]:
Interact.run(transform)

INFO:tensorflow:Assets written to: faisal_ag_037-pipeline\resto-pipeline\Transform\transform_graph\28\.temp_path\tftransform_tmp\320b1266815e4b55bb91584f0781bd85\assets


INFO:tensorflow:Assets written to: faisal_ag_037-pipeline\resto-pipeline\Transform\transform_graph\28\.temp_path\tftransform_tmp\320b1266815e4b55bb91584f0781bd85\assets


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 28
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

# Model Development

Model development adalah tahap pengembangan model Machine Learning, di mana model dilatih menggunakan data yang sudah diproses.

## Trainer

Trainer adalah komponen yang bertanggung jawab untuk melatih model Machine Learning dengan menggunakan data yang sudah diproses. Komponen ini mengimplementasikan algoritma pembelajaran, mengatur hyperparameter, dan menghasilkan model yang terlatih.

In [21]:
TRAINER_FILE = "resto-trainer.py"

In [22]:
%%writefile {TRAINER_FILE}

import tensorflow as tf
import tensorflow_transform as tft 
from tensorflow.keras import layers
import os  
import tensorflow_hub as hub
from tfx.components.trainer.fn_args_utils import FnArgs

LABEL = "label"
SENTENCE = "sentence"

def transformed_name(key):
    return key + "_xf"

def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, 
             tf_transform_output,
             num_epochs,
             batch_size=16)->tf.data.Dataset:
    
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())
    
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key = transformed_name(LABEL))
    return dataset

VOCAB_SIZE = 10000
SENTENCE_LENGTH = 75

vectorize_layer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SENTENCE_LENGTH)

embedding_dim=128
def model_builder():
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(SENTENCE), dtype=tf.string)
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name="embedding")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs = outputs)
    model.compile(
        loss = 'binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.01),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    
    )
    model.summary()
    
    return model 

def _get_serve_tf_examples_fn(model, tf_transform_output):
    model.tft_layer = tf_transform_output.transform_features_layer()
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL)
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        
        return model(transformed_features)
    return serve_tf_examples_fn
    
def run_fn(fn_args: FnArgs) -> None:
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )
    EarlyStopper = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', 
                                                    mode='max', 
                                                    verbose=1, 
                                                    patience=15)
    
    ModelCheck = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, 
                                                    monitor='val_binary_accuracy', 
                                                    mode='max', 
                                                    verbose=1, 
                                                    save_best_only=True)
    

    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)
    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(SENTENCE)]
                for i in list(train_set)]])
    
    # Build the model
    model = model_builder()
    
    
    # Train the model
    model.fit(x = train_set,
            validation_data = val_set,
            callbacks = [tensorboard_callback, EarlyStopper, ModelCheck],
            steps_per_epoch = 16, 
            validation_steps= 8,
            epochs=15)
    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                                    tf.TensorSpec(
                                    shape=[None],
                                    dtype=tf.string,
                                    name='examples'))
    }
    
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)


Overwriting resto-trainer.py


In [23]:
model_trainer  = Trainer(
    module_file=os.path.abspath(TRAINER_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schemaGen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)


In [24]:
Interact.run(model_trainer)

Instructions for updating:
Use `tf.data.Dataset.map(tf.io.parse_example(...))` instead.


Instructions for updating:
Use `tf.data.Dataset.map(tf.io.parse_example(...))` instead.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sentence_xf (InputLayer)    [(None, 1)]               0         
                                                                 
 tf.reshape (TFOpLambda)     (None,)                   0         
                                                                 
 text_vectorization (TextVe  (None, 75)                0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 75, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 128)               16512 

INFO:tensorflow:Assets written to: faisal_ag_037-pipeline\resto-pipeline\Trainer\model\29\Format-Serving\assets


16/16 [==============================] - 2s 100ms/step - loss: 0.7041 - binary_accuracy: 0.4766 - val_loss: 0.6927 - val_binary_accuracy: 0.5000
Epoch 2/15
14/16 [=========================>....] - ETA: 0s - loss: 0.7031 - binary_accuracy: 0.4286
Epoch 2: val_binary_accuracy did not improve from 0.50000
16/16 [==============================] - 0s 11ms/step - loss: 0.7017 - binary_accuracy: 0.4453 - val_loss: 0.6941 - val_binary_accuracy: 0.5000
Epoch 3/15
14/16 [=========================>....] - ETA: 0s - loss: 0.7002 - binary_accuracy: 0.5268
Epoch 3: val_binary_accuracy improved from 0.50000 to 0.51562, saving model to faisal_ag_037-pipeline\resto-pipeline\Trainer\model\29\Format-Serving
INFO:tensorflow:Assets written to: faisal_ag_037-pipeline\resto-pipeline\Trainer\model\29\Format-Serving\assets


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: faisal_ag_037-pipeline\resto-pipeline\Trainer\model\29\Format-Serving\assets


INFO:tensorflow:Assets written to: faisal_ag_037-pipeline\resto-pipeline\Trainer\model\29\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 29
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

# Analysis and Validating the Model

 Analisis dan validasi model adalah proses untuk mengevaluasi kinerja model dan memastikan bahwa model bekerja sesuai harapan sebelum digunakan dalam produksi.

## Resolver

Resolver digunakan untuk menentukan versi artefak yang sesuai (seperti data, model, atau skema) untuk digunakan dalam tahap pipeline berikutnya. Ini membantu dalam memilih artefak terbaik yang memenuhi kriteria tertentu untuk analisis lebih lanjut atau pelatihan ulang model.

In [25]:
modelResolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 

In [26]:
Interact.run(modelResolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 30
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## Evaluator

Evaluator adalah komponen yang digunakan untuk mengevaluasi kinerja model menggunakan metrik evaluasi standar (seperti akurasi, presisi, recall, dll.). Evaluator membantu dalam menentukan apakah model siap untuk di-deploy ke produksi atau perlu perbaikan lebih lanjut.

In [27]:
evalConfig = tfma.EvalConfig(
    model_specs= [tfma.ModelSpec(label_key='label')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]
 
)

In [28]:
modelEvaluator = Evaluator(
    examples=exampleGen.outputs['examples'],
    model=model_trainer.outputs['model'],
    baseline_model=modelResolver.outputs['model'],
    eval_config=evalConfig)
 

In [29]:
Interact.run(modelEvaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 31
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [30]:
IPython.get_ipython().run_line_magic('matplotlib', 'inline')

In [31]:
eval_result = modelEvaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
display(tfma.view.render_slicing_metrics(tfma_result))
display(tfma.addons.fairness.view.widget_view.render_fairness_indicator(tfma_result))

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

# Model Deployment

Model deployment adalah proses menempatkan model Machine Learning yang sudah dilatih ke dalam lingkungan produksi, di mana model dapat digunakan untuk prediksi.

## Pusher

Pusher adalah komponen yang digunakan untuk memindahkan model yang sudah divalidasi ke lingkungan produksi. Komponen ini memastikan bahwa model siap di-deploy dan dapat digunakan untuk membuat prediksi real-time atau batch dalam sistem produksi.

In [32]:
modelPusher = Pusher(
model=model_trainer.outputs['model'],
model_blessing= modelEvaluator.outputs['blessing'],
push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory='serving_model_dir/resto-sentiment-detection-model'))
 
)

In [33]:
Interact.run(modelPusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 32
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None)